## This notebook gets all the series IDs that are relevant by traversing the tree of categories and getting all the series IDs for each category in the FRED database.

In [1]:
import pandas as pd
import numpy as np
from fred import Fred
import csv
import datetime
import time

In [2]:
fr = Fred(api_key='d99c7df85ba716ab102c3b81c354d6fa',response_type='df')

In [31]:
# res is the result dataframe from calling the API
# lev1 is the all the category ids that are children to the root(0) category
res = fr.category.children(0)
lev1 = list(res.id)

In [41]:
# lev2 is the next level of categories
# if it doesn't have any children, then it is added to the cat_leaf list which will contain all the final categories
lev2 = []
cat_leaf = []
for cid in lev1:
    res = fr.category.children(cid)
    if not res.empty:
        lev2.extend(list(res.id))
    else:
        cat_leaf.append(cid)

In [44]:
lev3 = []
for cid in lev2[:32]:
    res = fr.category.children(cid)
    if not res.empty:
        lev3.extend(list(res.id))
    else:
        cat_leaf.append(cid)

In [47]:
for cid in lev2[32:]:
    res = fr.category.children(cid)
    if not res.empty:
        lev3.extend(list(res.id))
    else:
        cat_leaf.append(cid)

In [59]:
# had to add a sleep because it was making too many calls to the api
lev4=[]
count = 1
for cid in lev3:
    res = fr.category.children(cid)
    if not res.empty:
        lev4.extend(list(res.id))
    else:
        cat_leaf.append(cid)
    if (count % 50) == 0:
        time.sleep(60)
    count += 1

In [61]:
len(cat_leaf)

515

In [63]:
len(lev4)

542

In [64]:
lev5=[]
count = 1
for cid in lev4:
    res = fr.category.children(cid)
    if not res.empty:
        lev5.extend(list(res.id))
    else:
        cat_leaf.append(cid)
    if (count % 50) == 0:
        time.sleep(60)
    count += 1

In [65]:
len(cat_leaf)

860

In [66]:
len(lev5)

3880

In [67]:
lev6=[]
count = 1
for cid in lev5:
    res = fr.category.children(cid)
    if not res.empty:
        lev6.extend(list(res.id))
    else:
        cat_leaf.append(cid)
    if (count % 50) == 0:
        time.sleep(60)
    count += 1

In [68]:
len(cat_leaf)

4718

In [71]:
len(lev6)

24

In [72]:
lev7=[]
count = 1
for cid in lev6:
    res = fr.category.children(cid)
    if not res.empty:
        lev7.extend(list(res.id))
    else:
        cat_leaf.append(cid)
    if (count % 50) == 0:
        time.sleep(60)
    count += 1

In [73]:
len(cat_leaf)

4739

In [74]:
len(lev7)

11

In [75]:
lev8=[]
count = 1
for cid in lev7:
    res = fr.category.children(cid)
    if not res.empty:
        lev8.extend(list(res.id))
    else:
        cat_leaf.append(cid)
    if (count % 50) == 0:
        time.sleep(60)
    count += 1

In [76]:
len(cat_leaf)

4749

In [78]:
len(lev8)

1

In [79]:
lev8

[33259]

In [85]:
fr.category.children(33259)

,id,name,parent_id
0,33260,Total,33259


In [86]:
cat_leaf.append(33260)

In [87]:
# final number of categories is 4750
len(cat_leaf)

4750

In [91]:
with open('../data/raw/all_cat_ids.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(cat_leaf)

In [97]:
# get all the series ids for each category
# the max number of series is 1000 per release
# ser_ids is a list of all the series
# rel_2long contains categories with more than 1000 series, handle this later
params = {
         'limit':1000,
        'order_by':'observation_start',
         }
ser_ids = []
rel_2long = []
count = 1
for cat_id in cat_leaf:
    res = fr.category.series(cat_id,params=params)
    res = res[res.observation_start < '1968-12-31']
    if len(res) == 1000:
        rel_2long.append(cat_id)
    else:        
        if not res.empty:
            res = res[(res.observation_end > '2018-12-31') & (res.frequency != 'Annual')]
            ser_ids.extend(list(res.id))
    if (count % 60) == 0:
        time.sleep(60)
    count += 1
print(len(ser_ids), len(rel_2long), count)

AttributeError: 'DataFrame' object has no attribute 'observation_start'

In [105]:
cat_leaf[442]

33085

In [111]:
# sometimes you get errors and then have to continue downloading data at the failure point
count = 444
for cat_id in cat_leaf[443:]:
    res = fr.category.series(cat_id,params=params)
    if not res.empty:
        res = res[res.observation_start < '1968-12-31']
        if len(res) == 1000:
            rel_2long.append(cat_id)
        else:        
            if not res.empty:
                res = res[(res.observation_end > '2018-12-31') & (res.frequency != 'Annual')]
                if not res.empty:
                    ser_ids.extend(list(res.id))
    if (count % 60) == 0:
        time.sleep(60)
    count += 1
print(len(ser_ids), len(rel_2long), count)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1270-01-01 00:00:00

In [126]:
fr.category.details(33839)

,id,name,parent_id
0,33839,A Millennium of Macroeconomic Data for the UK,33060


In [138]:
# cat ids 33845, 33843, 33847 have out of bounds dates that fred can't handle
cat_leaf[508:519]

[33845, 33843, 33847, 33849, 33851, 33853, 33855, 287, 286, 285, 284]

In [139]:
# continue after errors
count = 512
for cat_id in cat_leaf[511:]:
    res = fr.category.series(cat_id,params=params)
    if not res.empty:
        res = res[res.observation_start < '1968-12-31']
        if len(res) == 1000:
            rel_2long.append(cat_id)
        else:        
            if not res.empty:
                res = res[(res.observation_end > '2018-12-31') & (res.frequency != 'Annual')]
                if not res.empty:
                    ser_ids.extend(list(res.id))
    if (count % 60) == 0:
        time.sleep(60)
    count += 1
print(len(ser_ids), len(rel_2long), count)

12371 5 4751


In [140]:
# initial download finished, these 5 categories contain more than 1000 series, so continue with offset
rel_2long

[21, 33021, 32281, 32267, 33835]

In [142]:
for cat_id in rel_2long:
    res = fr.category.details(cat_id)
    print(res.name)

0    Price Indexes & Deflators
Name: name, dtype: object
0    Quantity Indexes
Name: name, dtype: object
0    Japan
Name: name, dtype: object
0    United States
Name: name, dtype: object
0    Section 1. General Statistics of All Banks in ...
Name: name, dtype: object


In [149]:
params = {
         'limit':1000,
        'order_by':'observation_start',
        'offset':0,
         }

In [150]:
# get the first 1000 series of these categories that are 2 long
for cat_id in rel_2long:
    res = fr.category.series(cat_id,params=params)
    res = res[(res.observation_end > '2018-12-31') & (res.frequency != 'Annual')]
    ser_ids.extend(list(res.id))
len(ser_ids)

13997

In [151]:
params = {
         'limit':1000,
        'order_by':'observation_start',
        'offset':1000,
         }

In [152]:
# get the next 1000 series
cat_2long=[]
for cat_id in rel_2long:
    res = fr.category.series(cat_id,params=params)
    if not res.empty:
        res = res[res.observation_start < '1968-12-31']
        if len(res) == 1000:
            cat_2long.append(cat_id)          
        if not res.empty:
            res = res[(res.observation_end > '2018-12-31') & (res.frequency != 'Annual')]
            if not res.empty:
                ser_ids.extend(list(res.id))
print(len(ser_ids), len(cat_2long))

14535 0


In [153]:
# write all the series ids, total number of series is 14535!
with open('../data/raw/all_ser_ids_1969-2019.csv', 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(ser_ids)